# התחלת תרגיל

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
kviutdrivers = pd.read_csv('https://raw.githubusercontent.com/meitalbensinai/AlgoTrainingNoSol/master/Basic/Azrieli%20and%20Sons/data/Drivers_with_kviut.csv')

In [3]:
newDrivers = pd.read_csv('https://raw.githubusercontent.com/meitalbensinai/AlgoTrainingNoSol/master/Basic/Azrieli%20and%20Sons/data/new_drivers.csv')

לאחד את הטבלאות של הנהגים

In [4]:
drivers = kviutdrivers.append(newDrivers)

In [5]:
drivers.set_index('id' ,inplace= True)

להוריד שורה מיותרת

In [6]:
drivers.drop('Unnamed: 0', inplace = True, axis = 1)

In [7]:
drivers.sort_values('id', inplace = True)

לנקות מין

In [8]:
drivers['gender'] = np.where(drivers['gender'].isin(['M','m','boy','male']),'m',np.where(drivers['gender'].isin(['F','f','female','woman','girl']),'f','unknown'))

In [9]:
drivers.gender.unique()

array(['f', 'm', 'unknown'], dtype=object)

להפוך תאריכי לידה לתאריך

In [10]:
drivers.birthdate =  pd.to_datetime(drivers.birthdate)

למלא ערכים חסרים של הגיל (ע"פ הותק)

In [11]:
vmean = drivers.vetek.mean()

In [12]:
bmean = drivers.birthdate.mean()

In [13]:
birthmean = bmean.year + bmean.month/12

In [14]:
agePerVetek = ((datetime.datetime.now().year-bmean.year)+(datetime.datetime.now().month-bmean.month)/12)/vmean

In [15]:
def getY(x):
    return np.floor(x.days/36.5)/10

In [16]:
drivers['age'] = (datetime.datetime.now() - drivers.birthdate)
drivers.age = drivers.age[drivers.age.notna()].apply(getY)
drivers.age[drivers.age.isna()] = drivers.vetek[drivers.age.isna()]*agePerVetek
drivers.drop('birthdate', axis = 1, inplace = True)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


להחליף את השם של החברות

In [17]:
gvia = pd.read_csv('https://raw.githubusercontent.com/meitalbensinai/AlgoTrainingNoSol/master/Basic/Azrieli%20and%20Sons/data/taarif.csv')

In [18]:
gvia.iloc[[3,4,7],0] = ['aminach','yes','hot']

In [19]:
gvia.dropna(how='all', inplace = True)

למלאות את הערכים החסרים

In [20]:
gvia.basic_taarif.fillna(gvia.basic_taarif.mean(), inplace = True)

 למלא את התעריף האקסטרה ע"י החישוב ממוצע פי כמה גדול תעריף האקסטרה כפול התעריף הרגיל של החברה

In [21]:
gvia.extra_milage.fillna(gvia.extra_milage.mean()/gvia.basic_taarif.mean()*gvia.basic_taarif, inplace = True)

 ללילה ולסופ"ש לקחתי את החציון כי ישנם ערכים גדולים משמעותית והממוצע לא יגיד הרבה

In [22]:
gvia.night_bonus.fillna(gvia.night_bonus.median(), inplace = True)
gvia.weekend_bonus.fillna(gvia.weekend_bonus.median(), inplace = True)
gvia

,customer,basic_taarif,extra_milage,night_bonus,weekend_bonus
1,jerusalem_muni,8.000000,40.000000,50.0,30.0
2,intel,8.000000,6.000000,20.0,0.0
3,aminach,15.000000,4.000000,0.0,25.0
4,yes,10.000000,8.000000,0.0,25.0
5,tnuva,6.000000,3.000000,80.0,0.0
6,angel,1.000000,5.000000,100.0,0.0
7,hot,7.000000,7.000000,10.0,40.0
8,osem,12.000000,4.000000,20.0,0.0
9,iaf,20.000000,10.000000,200.0,500.0
10,tara,6.000000,6.000000,50.0,30.0


לנקות את זמני הסיום של הנהג

In [23]:
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

In [24]:
summ = drivers

In [25]:
summ['Total_income'] = 0

In [26]:
summ['Total_km'] = 0

In [27]:
monthSumm = {}

In [28]:
for m in month:
    s = summ.copy()
    s['month'] = m
    monthSumm[m] = s

פונקציית עזר לחישוב משכורת

In [29]:
def slr(x):
    precent = 1
    if x.start_time.hour > 20:
        precent = precent + x.night_bonus/100
    if x.start_time.weekday() in [4,5]:
        precent = precent + x.weekend_bonus/100
    if x.km > 200:
        return (200 * x.basic_taarif + (x.km-200) * x.extra_milage)*precent
    return x.km * x.basic_taarif * precent

חישוב בלולאה על כל הקבצים

In [30]:
http = 'https://raw.githubusercontent.com/meitalbensinai/AlgoTrainingNoSol/master/Basic/Azrieli%20and%20Sons/data/trips_data/'
number = '%202015_478'
csv = '.csv'

In [ ]:
for m in month:
    print(m)
    for i in range(1353,1853):
        url = http + m + number + (str)(i) + csv
        df = pd.read_csv(url)

        df.drop('Unnamed: 0', inplace = True, axis=1)

        df.start_time = pd.to_datetime(df.start_time)

        df = df.merge(gvia)

        df['salary'] = df.apply(slr,axis=1)

        drvSlr = df.loc[:, ['driver_id', 'salary','km']]
        drvSlr.columns = ['id','salary','km']

        for i in drvSlr.id:
            monthSumm[m].Total_income[i] += drvSlr.salary[drvSlr.id == i].iloc[0]
            monthSumm[m].Total_km[i] += drvSlr.km[drvSlr.id == i].iloc[0]

january


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
summary = pd.DataFrame()

In [ ]:
for s in monthSumm:
    summary = summary.append(monthSumm[s])

In [ ]:
summary